## Import Package

In [1]:
# Base
from time import time
from datetime import timedelta
import random
import numpy as np
import pandas as pd

# Graph
import networkx as nx
# node embedding
from node2vec import Node2Vec
# sklearn measure
from sklearn import metrics

# Pytorch
import torch
from torch import nn
from torch.nn import init
from torch.optim import Adam
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader

In [2]:
class GraphStructure():   
    def __init__(self, G):
          self.G = G

    '''calucate disconnected pairs for negative sample'''
    def disconnected_node_pairs(self, node_list):
        possible_node_pairs = list()
        adjacency_matrix = nx.to_numpy_array(self.G, nodelist=node_list)
        for i in range(adjacency_matrix.shape[0]):
            for j in range(adjacency_matrix.shape[1]):
                if i != j:
                    try:
                        n = nx.shortest_path_length(G, str(i), str(j))
                    except:
                        n = 0
                    if n <= 2 and adjacency_matrix[i, j] == 0:
                        possible_node_pairs.append((node_list[i], node_list[j]))
#                 if i != j and adjacency_matrix[i][j] == 0:
#                     possible_node_pairs.append((node_list[i], node_list[j]))
        return possible_node_pairs

    '''calucate removable pairs for positive sample'''
    def removable_node_pairs(self, node_pairs_df):
        # check whether removing a node pair will cause
        # 1: graphic segmentation
        # 2: reduce the number of nodes
        removable_links_index = list()
        original_node_num = self.G.number_of_nodes()
        temp_node_pairs_df = node_pairs_df.copy()
        for i in tqdm(node_pairs_df.index.values):
            temp_G = nx.from_pandas_edgelist(temp_node_pairs_df.drop(index = i), "node1", "node2", create_using=nx.Graph())
            if (nx.number_connected_components(temp_G) == 1) and (temp_G.number_of_nodes() == original_node_num):
                removable_links_index.append(i)
                temp_node_pairs_df = temp_node_pairs_df.drop(index = i) 
        return removable_links_index

def load_dataset(file_path, split_symbol, read_title=False):
    node_pairs = list()
    with open(file_path, 'r') as f:
        if read_title:
            title = f.readline()
        for line in f.readlines():
            node_pairs.append(list(line.strip().split(split_symbol)))
        dataset_df = pd.DataFrame(node_pairs, columns=['node1', 'node2'])
    return dataset_df

def preprocess(node_pairs_df):
    instances = list()
    for i, row in node_pairs_df.iterrows():
        s_index, t_index, label = row
        instance = {
            'source': torch.LongTensor(np.array([int(s_index)])),
            'target': torch.LongTensor(np.array([int(t_index)])),
            'label': torch.FloatTensor(np.array([float(label)]))
        }
        instances.append(instance)
    return instances

## Load data

In [19]:
if __name__ == '__main__':
    # Random seed
    seed = 42
    test_sample_ratio = 0.2
    sample_rate = 1
    random.seed(seed)
    torch.cuda.manual_seed(seed)

    node_pairs_df = load_dataset('out.dimacs10-polblogs', split_symbol='\t', read_title=True)

## Dataset Splitting and Labeling

In [20]:
    # node_pairs = [ pair for pair in zip(node_pairs_df['node1'], node_pairs_df['node2'])]
    last_snapshot = nx.from_pandas_edgelist(node_pairs_df, 'node1', 'node2', create_using=nx.Graph())
    last_node_pairs_df = pd.DataFrame(list(last_snapshot.edges()), columns=['node1', 'node2']) 
    print('total # of nodes:', last_snapshot.number_of_nodes())
    print('total # of edges:', last_snapshot.number_of_edges())

total # of nodes: 1224
total # of edges: 16715


#### Testing data

In [21]:
    # Top 20% edges for test positive sample
    test_positive_num = int(last_snapshot.number_of_edges()*test_sample_ratio)
    test_positive_df = last_node_pairs_df.tail(test_positive_num).copy()
    
    # calculate unlink node pairs for test negative sample
    test_gs = GraphStructure(last_snapshot)
    test_no_edge_pairs = test_gs.disconnected_node_pairs(list(dict.fromkeys(last_node_pairs_df['node1'].to_list()+last_node_pairs_df['node2'].to_list())))
    test_no_edge_pairs_df = pd.DataFrame(test_no_edge_pairs, columns=['node1', 'node2'])
    test_negative_df = test_no_edge_pairs_df
    
    # labeling
    test_negative_df['label'] = 0
    test_positive_df['label'] = 1
    print("test # of negative: %d\t# of positive: %d" % (len(test_negative_df), len(test_positive_df)))
    
    test_negative_df = test_negative_df.sample(int(len(test_positive_df)*sample_rate), replace=True)
    test_dataset_df = test_negative_df.append(test_positive_df)
    test_negative_num, test_positive_num = test_dataset_df.label.value_counts()
    print("sample after:\n# of negative: %d\t# of positive: %d\n" % (test_positive_num, test_negative_num))
    print(test_dataset_df)

test # of negative: 1463522	# of positive: 3343
sample after:
# of negative: 3343	# of positive: 3343

        node1 node2  label
1341202   781   831      0
630683    734  1164      0
1036877  1102     9      0
1174063  1126   742      0
1306388  1113   465      0
...       ...   ...    ...
16710    1091  1161      1
16711    1117  1157      1
16712    1168  1210      1
16713    1180  1181      1
16714    1189  1213      1

[6686 rows x 3 columns]


#### Training data

In [22]:
    # Top 80% edges for train positive edge
    train_positive_num = last_snapshot.number_of_edges()-test_positive_num
    train_positive_df = test_positive_df.head(train_positive_num).copy()
    train_snapshot = nx.from_pandas_edgelist(train_positive_df, 'node1', 'node2', create_using=nx.Graph())
    
    # remove edges with test positive sample for training snapshot
    train_snapshot = last_snapshot.copy()
    for pair in zip(test_positive_df['node1'], test_positive_df['node2']):
        train_snapshot.remove_edge(*pair)
    
    # calculate unlink node pairs for train negative sample
    train_gs = GraphStructure(train_snapshot)
    train_no_edge_pairs = train_gs.disconnected_node_pairs(list(dict.fromkeys(train_positive_df['node1'].to_list()+train_positive_df['node2'].to_list())))
    train_no_edge_pairs_df = pd.DataFrame(train_no_edge_pairs, columns=['node1', 'node2'])
    train_negative_df = train_no_edge_pairs_df
    
    # labeling
    train_negative_df['label'] = 0
    train_positive_df['label'] = 1
    print("# of negative: %d\t# of positive: %d" % (len(train_negative_df), len(train_positive_df)))

    train_negative_df = train_negative_df.sample(len(train_positive_df), replace=True)
    train_dataset_df = train_negative_df.append(train_positive_df)
    train_positive_num, train_negative_num = train_dataset_df.label.value_counts()
    print("sample after:\n# of negative: %d\t# of positive: %d\n" % (train_positive_num, train_negative_num))
    print(train_dataset_df)

# of negative: 270896	# of positive: 3343
sample after:
# of negative: 3343	# of positive: 3343

       node1 node2  label
214392   607   823      0
181323   981  1030      0
46588    664  1117      0
138207   937  1125      0
105376   855  1008      0
...      ...   ...    ...
16710   1091  1161      1
16711   1117  1157      1
16712   1168  1210      1
16713   1180  1181      1
16714   1189  1213      1

[6686 rows x 3 columns]


#### Preprocessing

In [7]:
    test_instances = preprocess(test_dataset_df)
    train_instances = preprocess(train_dataset_df)
    
    print('# of test instances:', len(test_instances))
    print('# of train instances:', len(train_instances))
    print('# of total instances:', (len(train_instances)+len(test_instances)))
#     test_instances = preprocess(test_dataset_df)
#     valid_instances = preprocess(valid_dataset_df)
#     train_instances = preprocess(train_dataset_df)
    
#     print('# of test instances:', len(test_instances))
#     print('# of valid instances:', len(valid_stances))
#     print('# of train instances:', len(train_instances))
#     print('# of total instances:', (len(train_instances)+len(valid_instances)+len(test_instances)))

# of test instances: 840
# of train instances: 840
# of total instances: 1680


## Graph Node Embedding with Node2Vec

In [8]:
    node2vec = Node2Vec(train_snapshot, dimensions=128, walk_length=80, num_walks=10)

Generating walks (CPU: 1):   0%|          | 0/10 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:12<00:00,  1.27s/it]


In [9]:
    n2v_model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [10]:
    node_embedding = n2v_model.wv.vectors
    node_embedding.shape

(620, 128)

## Training

In [11]:
    class NodePairDataset(Dataset):
        def __init__(self, instances):
            self.instances = instances

        def __len__(self):
            return len(self.instances)

        def __getitem__(self, i):
            instance = self.instances[i]
            source = instance['source']
            target = instance['target']
            label = instance['label']
            return source, target, label
        
    def collate_fn(batch):
        source, target, labels = zip(*batch)
        source = torch.stack(source)
        target = torch.stack(target)
        labels = torch.stack(labels)
        return source, target, labels

    def get_dataloader(instances, collate_fn=collate_fn,batch_size=1, num_workers=2):
        dataset = NodePairDataset(instances)
        dataloader = DataLoader(dataset, collate_fn=collate_fn, shuffle=True, batch_size=batch_size, num_workers=num_workers)
        return dataloader

In [12]:
    class LinkEmbedding(nn.Module):
        def __init__(self, inputs_dim, output_dim):
            super(LinkEmbedding, self).__init__()
            self.weight = nn.Parameter(nn.init.xavier_uniform_(torch.empty(inputs_dim, output_dim)))
            
            
        def forward(self, hidden_state, source, target):
            propagation = torch.mul(hidden_state[source, :], hidden_state[target, :])
            propagation = propagation.matmul(self.weight)
            return propagation
    
    class GraphConvolution(nn.Module):
        def __init__(self, inputs_dim, hidden_dim):
            super(GraphConvolution, self).__init__()
            self.weight = nn.Parameter(nn.init.kaiming_normal_(torch.empty(inputs_dim, hidden_dim), mode='fan_in', nonlinearity='relu'))
            # self.weight = nn.Parameter(nn.init.xavier_uniform_(torch.empty(inputs_dim, hidden_dim)))
            
            
        def forward(self, input_features, adj_matrix):
            # aggregate 
            aggregate  = torch.mm(input_features, self.weight)
            propagation = torch.mm(adj_matrix, aggregate)
            return propagation
        
    class GCN(nn.Module):
        def __init__(self, inputs_dim, hidden_dim, output_dim):
            super(GCN, self).__init__()
            self.gcn_layer1 = GraphConvolution(inputs_dim, hidden_dim)
            self.gcn_layer2 = GraphConvolution(hidden_dim, hidden_dim)
            self.link_embed_layer = LinkEmbedding(hidden_dim, output_dim)
            self.relu = nn.ReLU()
            self.sigmoid = nn.Sigmoid()

        def forward(self, input_features, adj_matrix, source, target):
            hidden_state = self.relu(self.gcn_layer1(input_features, adj_matrix))
            hidden_state = self.gcn_layer2(hidden_state, adj_matrix)
            hidden_state = self.link_embed_layer(hidden_state, source, target)
            return hidden_state

In [13]:
    class GCNTrainer():
        def __init__(self, features, adj_matrix, train_instances, valid_instances=None, test_instances=None, 
            hidden_dim=16, epoch=1, max_patience=0, learning_rate=1e-2, batch_size=1,num_workers=2, valid=False):

            # parameters
            self.valid = valid
            self.epochs = epoch
            self.learning_rate = learning_rate
            self.batch_size = batch_size
            self.num_workers = num_workers
            # early stop
            self.best_valid_loss = 1e10
            self.max_patience = max_patience
            self.patience = 0

            # setup cuda device
            self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

            # dataset
            self.train_instances = train_instances
            self.valid_instances = valid_instances
            self.test_instances = test_instances
            self.features = torch.FloatTensor(features).cuda()
            self.adj_matrix = torch.FloatTensor(self.normalize(adj_matrix)).cuda()
            
            # GCN Model
            self.model = GCN(self.features.shape[1], hidden_dim, output_dim=1)
            self.model.cuda()
            # print(self.model)

            # Adam optimizer with hyper-parameter
            # self.optimizer = SGD(self.model.parameters(), lr=self.learning_rate)
            self.optimizer = Adam(self.model.parameters(), lr=self.learning_rate)

            # Binary Cross Entropy with Loss for criterion
            self.criterion = nn.BCEWithLogitsLoss()
        

        def normalize(self, A):
            '''
            :var I: identity matrix
            :var A: adjacency matrix
            :var D: degree matrix
            :var A_hat: adding self-loops
            :var D_inv: degree inverse matrix
            '''
            I = np.matrix(np.identity(A.shape[0]))
            A_hat = I + A
            
            D = np.array(np.sum(A, axis=0))
            D_inv = D**-0.5
            D_inv[np.isinf(D_inv)] = 0.
            D_inv = np.diag(D_inv)

            A_hat = D_inv * A_hat * D_inv
            return A_hat
        
        def accuracy(self, predicts, labels):
            predicts_labels = torch.round(torch.sigmoid(predicts))
            total_correct = (predicts_labels == labels).sum().float()
            return torch.round((total_correct / labels.shape[0]) * 100)

        def train(self):
            start_time = time()
            self.optimizer.zero_grad()

            train_dataloader = get_dataloader(self.train_instances, collate_fn=collate_fn, batch_size=self.batch_size, num_workers=self.num_workers)
            for epoch in range(self.epochs):
                self.model.train()
                epoch_loss, epoch_acc = 0, 0
                ''' train '''
                for i, batch in enumerate(train_dataloader, start=1):
                    batch = (tensor.cuda() for tensor in batch)
                    source, target, labels = batch
                    # forward
                    # feature: all node embedding
                    outputs = self.model(self.features, self.adj_matrix, source, target)
                    outputs = outputs.reshape(labels.size())
                    # backward
                    loss = self.criterion(outputs, labels)
                    acc = self.accuracy(outputs, labels)
                    epoch_loss += loss.item()
                    epoch_acc += acc
                    
                    loss.backward()
                    # optimize
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    
                    # Progressbar
                    elapsed_time = time() - start_time
                    elapsed_time = timedelta(seconds=int(elapsed_time))
                    # print("Epoch %d/%d | loss: %.6f | acc: %f | batch: [%d/%d] | %s" % (epoch+1, self.epochs, loss, acc, i, len(train_dataloader), elapsed_time))
                
                print("Epoch %d/%d - train_loss: %.6f - train_acc: %.2f%%" 
                      % (epoch+1, self.epochs, epoch_loss/len(train_dataloader), epoch_acc/len(train_dataloader)))
                
                ''' validate '''
                if self.valid:
                    valid_loss, valid_acc = self.validate()
                    elapsed_time = time() - start_time
                    elapsed_time = timedelta(seconds=int(elapsed_time))
                    print("Epoch %d/%d - valid_loss: %.6f - valid_acc: %.2f%%" % (epoch+1, self.epochs, valid_loss, valid_acc))

                    # early stoping
                    if valid_loss < self.best_valid_loss:
                        self.patience = 0
                        self.best_valid_loss = valid_loss
                    else:
                        self.patience += 1

                    if self.patience > self.max_patience:
                        print('Earlystop at epoch %d' % (epoch+1))
                        break


        def validate(self):
            total_loss, total_acc = 0, 0
            self.model.eval()
            with torch.no_grad():
                valid_dataloader = get_dataloader(self.valid_instances, collate_fn=collate_fn, batch_size=self.batch_size, num_workers=self.num_workers)
                for batch in valid_dataloader:
                    batch = (tensor.cuda() for tensor in batch)
                    source, target, labels = batch
                    outputs = self.model(self.features, self.adj_matrix, source, target)
                    outputs = outputs.reshape(labels.size())
                    loss = self.criterion(outputs, labels)
                    # loss and accuracy
                    total_loss += loss.item()
                    total_acc += self.accuracy(outputs, labels)
            
            total_loss /= len(valid_dataloader)
            total_acc /= len(valid_dataloader)
            return float(total_loss), float(total_acc)

        def test(self):
            total_loss, total_acc, auc = 0, 0, 0
            total_predicts, total_labels = list(), list()
            self.model.eval()
            with torch.no_grad():
                test_dataloader = get_dataloader(self.test_instances, collate_fn=collate_fn, batch_size=self.batch_size, num_workers=self.num_workers)
                for batch in test_dataloader:
                    batch = (tensor.cuda() for tensor in batch)
                    source, target, labels = batch
                    outputs = self.model(self.features, self.adj_matrix, source, target)
                    outputs = outputs.reshape(labels.size())
                    # auc
                    total_predicts += torch.round(torch.sigmoid(outputs.cpu())).squeeze().numpy().tolist()
                    total_labels += torch.round(torch.sigmoid(labels.cpu())).squeeze().numpy().tolist()
                    loss = self.criterion(outputs, labels)
                    # loss and accuracy
                    total_loss += loss.item()
                    total_acc += self.accuracy(outputs, labels)
            
            total_loss /= len(test_dataloader)
            total_acc /= len(test_dataloader)
            
            fpr, tpr, thresholds = metrics.roc_curve(total_labels, total_predicts, pos_label=1)
            auc = metrics.auc(fpr, tpr)
            return float(total_loss), float(total_acc), auc

In [14]:
    adj_matrix = nx.to_numpy_array(train_snapshot)
    print(adj_matrix)

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
    trainer = GCNTrainer(features=node_embedding, adj_matrix=adj_matrix, 
                         train_instances=train_instances, 
                         valid_instances=None,
                         test_instances=test_instances,
                         hidden_dim=64, epoch=300,learning_rate=1e-2, batch_size=128, num_workers=2, valid=False)

<ipython-input-13-1386f989da06>:51: RuntimeWarning: divide by zero encountered in power
  D_inv = D**-0.5


In [16]:
    trainer.train()

Epoch 1/300 - train_loss: 0.739812 - train_acc: 48.29%
Epoch 2/300 - train_loss: 0.686383 - train_acc: 53.86%
Epoch 3/300 - train_loss: 0.681158 - train_acc: 57.43%
Epoch 4/300 - train_loss: 0.667159 - train_acc: 58.43%
Epoch 5/300 - train_loss: 0.674555 - train_acc: 57.29%
Epoch 6/300 - train_loss: 0.672268 - train_acc: 54.00%
Epoch 7/300 - train_loss: 0.660025 - train_acc: 58.71%
Epoch 8/300 - train_loss: 0.652384 - train_acc: 58.43%
Epoch 9/300 - train_loss: 0.640271 - train_acc: 59.71%
Epoch 10/300 - train_loss: 0.634239 - train_acc: 58.86%
Epoch 11/300 - train_loss: 0.639702 - train_acc: 59.86%
Epoch 12/300 - train_loss: 0.635729 - train_acc: 59.86%
Epoch 13/300 - train_loss: 0.621265 - train_acc: 63.57%
Epoch 14/300 - train_loss: 0.607730 - train_acc: 61.29%
Epoch 15/300 - train_loss: 0.595553 - train_acc: 62.14%
Epoch 16/300 - train_loss: 0.581287 - train_acc: 64.00%
Epoch 17/300 - train_loss: 0.578854 - train_acc: 63.00%
Epoch 18/300 - train_loss: 0.567964 - train_acc: 66.86%
E

In [18]:
    loss, accuracy, auc = trainer.test()
    print('test_loss:%.6f | test_acc:%.2f | auc:%.2f' % (loss, accuracy, auc))

test_loss:4.305449 | test_acc:68.57 | auc:0.68
